In [1]:
import os
import pandas as pd
print (os.environ['CONDA_DEFAULT_ENV'])

pytorch_env


In [2]:
from transformers import pipeline

In [23]:
nlp = pipeline("sentiment-analysis")

In [4]:
result = nlp("ugly photo")[0]

In [5]:
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: NEGATIVE, with score: 0.9998


In [3]:
import instaloader
PATH_DATA = 'posts/'
MAX_COMMENT = 50

def init_insta(username=None, password=None, session_user=None):
    L = instaloader.Instaloader(
        dirname_pattern = "posts/{profile}/{date}")
    if session_user is None and username and password is not None:
        print('logging in ... ')
        L.login(username, password)
        print('logged in -> ', username)
    else:
        L.load_session_from_file(session_user)
    
    return L

In [4]:
def set_profile(L, username_profile):
    profile = instaloader.Profile.from_username(L.context, username_profile)
    return profile

In [8]:
def get_profile_data(profile):
    if profile.is_private == True:
        print('PRIVINFO NOT AVAILABLE')
        return
    
    temp_vect = []
    
    try:
        temp_has_public_story = profile.has_public_story
        temp_has_viewable_story = profile.has_viewable_story
    except Exception as ex:
        print("EXECPTION -->", ex)
        temp_has_public_story = None
        temp_has_viewable_story = None
    dict_profile_data = {
        'user_id': profile.userid,
        'username': profile.username,
        'followed_by_viewer': profile.followed_by_viewer,
        'post_count': profile.mediacount,
        'igtv_count': profile.igtvcount,
        'n_follower': profile.followers,
        'n_followees': profile.followees,
        'external_url': profile.external_url,
        'is_bussines': profile.is_business_account,
        'business_Category': profile.business_category_name,
        'biography': profile.biography,
        'blocked_by_viewer': profile.blocked_by_viewer,
        'follows_viewer': profile.follows_viewer,
        'full_name': profile.full_name,
        'has_blocked_viewer': profile.has_blocked_viewer,
        'has_public_story': temp_has_public_story,
        'has_viewable_story': temp_has_viewable_story,
        'has_requested_viewer': profile.has_requested_viewer,
        'is_verified': profile.is_verified,
        'requested_by_viewer': profile.requested_by_viewer,
        'profile_pic_url': profile.profile_pic_url,
        'has_higlighted_reels': profile.has_highlight_reels,
        'followed_by_viewer': profile.followed_by_viewer
    }
    temp_vect.append(dict_profile_data)
    profile_data = pd.DataFrame(temp_vect)

    return profile_data

In [5]:
# GET DATA POST FROM SHORTCODE
def get_post_from_shortcode(SHORTCODE:str, MAX_COMMENT:int):
    post = Post.from_shortcode(L.context, SHORTCODE)
    try:
        accessibility_caption = str(post._asdict()['accessibility_caption'])
    except Exception as ex:
        print(ex)
        accessibility_caption = None
        #INFORMATION OF THE POST GOING INTO THE CSV
        post_info_dict = {
            'title': post.title,
            'owner_username': post.owner_username,
            'date_and_time': post.date,
            'type_of_post': post.typename,
            'mediacount': post.mediacount,
            'caption': post.caption,
            'n_caption_hashatags': len(post.caption_hashtags),
            'caption_hashtags': post.caption_hashtags,
            'n_mentions_post': len(post.caption_mentions),
            'n_tagged_users': len(post.tagged_users),
            'is_video': post.is_video,
            'n_likes': post.likes,
            'n_comments': post.comments,
            'is_sponsored': post.is_sponsored,
            'url': 'instagram.com/p/{}/'.format(post.shortcode),
            'description_of_post': accessibility_caption,
            'has_image_on_dataset': image_presence, 
            'comment_path': path_comments_csv
        }
        comments_vect = []
        comment_count = 0
        for comment in post.get_comments():
            answer_count = 0
            for answer in comment.answers:
                answer_count += 1
            comment_info_dict = {
                'date_and_time': comment.created_at_utc,
                'profile': comment.owner.username,
                'text': comment.text,
                'n_likes': comment.likes_count,
                'answer_count': answer_count
            }
            
            comments_vect.append(comment_info_dict)
            if comment_count == MAX_COMMENT:
                break
            comment_count += 1
            print('.', end='')
        print('End Comments')
        comment_df = pd.DataFrame(comments_vect)
        post_df = pd.DataFrame([post_info_dict])

        return post_df, comment_df

    

In [21]:
#GET DATA OF THE PROFILE
def get_post_and_comment(MAX_COMMENT: int, L = None, MAX_POST=5, profile=None):
        
        counter_post = 1
        post_profile = {"profile": profile.username, 'posts': []}
        for post in profile.get_posts():
            comments_vect = []
            try:
                accessibility_caption = str(
                    post._asdict()['accessibility_caption'])
            except Exception as ex:
                print(ex)
                accessibility_caption = None
            #INFORMATION OF THE POST GOING INTO THE CSV
            post_info_dict = {
                'title': post.title,
                'owner_username': post.owner_username,
                'date_and_time': post.date,
                'type_of_post': post.typename,
                'mediacount': post.mediacount,
                'caption': post.caption,
                'n_caption_hashatags': len(post.caption_hashtags),
                'caption_hashtags': post.caption_hashtags,
                'n_mentions_post': len(post.caption_mentions),
                'n_tagged_users': len(post.tagged_users),
                'is_video': post.is_video,
                'n_likes': post.likes,
                'n_comments': post.comments,
                'is_sponsored': post.is_sponsored,
                'url_link' : post.url,
                'url_insta': 'instagram.com/p/{}/'.format(post.shortcode),
                'description_of_post': accessibility_caption,
            }

            # DOWNLOAD AND STORE COMMENT
            print('Start Comments')

            comment_count = 0
            for comment in post.get_comments():
                answer_count = 0
                """
                for answer in comment.answers:
                    answer_count += 1
                    if answer_count == 5:
                        break
                """
                analisys, score = nlp(
                    str(comment.text).strip())
                comment_info_dict = {
                    'date_and_time': comment.created_at_utc,
                    'profile': comment.owner.username,
                    'text': str(comment.text).strip(),
                    'n_likes': comment.likes_count,
                   # 'answer_count': answer_count,
                    'sentiment_analysis': analisys,
                    'score': score
                }


                comments_vect.append(comment_info_dict)
                if comment_count == MAX_COMMENT:
                    print("MAX COMMENT")
                    break
                comment_count += 1
                print('.', end='')
            print('End Comments')

            #L.download_pic(path_pic_jpg, post.url, post.date_utc)

            #STORING DATA SCRAPED AND UPLOAD RELATIVE CSVs
            comment_df = pd.DataFrame(comments_vect)
            post_df = pd.DataFrame([post_info_dict])

            post = {'post_info': post_df, 'comments': comment_df}
            post_profile['posts'].append(post)
            print("END__POST")
            #IF MAX POST DOWNLOADED BREAK
            if counter_post % MAX_POST == 0:
                print('Post Reached')
                break
            counter_post += 1

        return post_profile

In [6]:
scraper = init_insta(username='test_lorenz', password='provaprova')
profile = set_profile(L = scraper, username_profile="bbcnews")

logging in ... 
logged in ->  test_lorenz


In [9]:
get_profile_data(profile)

,user_id,username,followed_by_viewer,post_count,igtv_count,n_follower,n_followees,external_url,is_bussines,business_Category,...,follows_viewer,full_name,has_blocked_viewer,has_public_story,has_viewable_story,has_requested_viewer,is_verified,requested_by_viewer,profile_pic_url,has_higlighted_reels
0,16278726,bbcnews,False,12589,2433,17784034,4,https://bbc.in/NewsInsta,True,Publishers,...,False,BBC News,False,True,True,False,True,False,https://scontent-lhr8-1.cdninstagram.com/v/t51...,True


In [24]:
post_profile = get_post_and_comment(50,L = scraper,MAX_POST = 50,profile= profile )

Start Comments


ValueError: not enough values to unpack (expected 2, got 1)

In [27]:
import numpy as np

In [28]:
df_comments = post_profile['posts']
np.datetime_as_string(post['post_info']['date_and_time'].values[0])

'2021-04-27T12:59:56.000000000'

In [15]:
for post in post_profile['posts']:
    print(type(post['comments']))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
